In [1]:
import socket
import spynnaker8 as p
from pyNN.random import NumpyRNG, RandomDistribution
from pyNN.utility import Timer
from pyNN.utility.plotting import Figure, Panel
import matplotlib.pyplot as plt

# === Define parameters ===
threads = 1
rngseed = 98766987
parallel_safe = True

n = 1500          # number of cells
r_ei = 4.0        # number of excitatory cells:number of inhibitory cells
pconn = 0.02      # connection probability
stim_dur = 50.    # (ms) duration of random stimulation
rate = 100.       # (Hz) frequency of the random stimulation

dt = 1          # (ms) simulation timestep
tstop = 1000      # (ms) simulaton duration
delay = 2

# Cell parameters
area = 20000.     # (µm²)
tau_m = 20.       # (ms)
cm = 1.           # (µF/cm²)
g_leak = 5e-5     # (S/cm²)

E_leak = None
E_leak = -60.  # (mV)
v_thresh = -50.   # (mV)
v_reset = -60.    # (mV)
t_refrac = 5.     # (ms) (clamped at v_reset)
v_mean = -60.     # (mV) mean membrane potential, for calculating CUBA weights
tau_exc = 5.      # (ms)
tau_inh = 10.     # (ms)

# Synapse parameters
Gexc = None
Ginh = None
Gexc = 4.     # (nS)
Ginh = 51.    # (nS)
Erev_exc = 0.     # (mV)
Erev_inh = -80.   # (mV)

# === Calculate derived parameters ===
area = area * 1e-8                     # convert to cm²
cm = cm * area * 1000                  # convert to nF
Rm = 1e-6 / (g_leak * area)            # membrane resistance in MΩ
assert tau_m == cm * Rm                # just to check

n_exc = int(round((n * r_ei / (1 + r_ei))))  # number of excitatory cells
n_inh = n - n_exc                            # number of inhibitory cells

print("{} {}".format(n_exc, n_inh))

w_exc = None
w_inh = None
celltype = None
celltype = p.IF_cond_exp
w_exc = Gexc * 1e-3              # We convert conductances to uS
w_inh = Ginh * 1e-3


# === Build the network ===

node_id = p.setup(
    timestep=dt, 
    min_delay=delay, 
    max_delay=delay)

p.set_number_of_neurons_per_core(p.IF_curr_exp, 100)      # this will set
#  100 neurons per core
p.set_number_of_neurons_per_core(p.IF_cond_exp, 50)      # this will set
# 50 neurons per core
np = 1

host_name = socket.gethostname()

cell_params = {'tau_m': tau_m,
               'tau_syn_E': tau_exc,
               'tau_syn_I': tau_inh,
               'v_rest': E_leak,
               'v_reset': v_reset,
               'v_thresh': v_thresh,
               'cm': cm,
               'tau_refrac': t_refrac,
               'i_offset': 0
               }

print(cell_params)

cell_params['e_rev_E'] = Erev_exc
cell_params['e_rev_I'] = Erev_inh

exc_cells = p.Population(
    n_exc, celltype(**cell_params), label="Excitatory_Cells")
inh_cells = p.Population(
    n_inh, celltype(**cell_params), label="Inhibitory_Cells")
exc_conn = None
ext_stim = None
ext_stim = p.Population(
    20, p.SpikeSourcePoisson(rate=rate, duration=stim_dur),
    label="expoisson")
rconn = 0.01
ext_conn = p.FixedProbabilityConnector(rconn)
ext_stim.record("spikes")

rng = NumpyRNG(seed=rngseed, parallel_safe=True)
uniformDistr = RandomDistribution('uniform', [v_reset, v_thresh], rng=rng)
exc_cells.initialize(v=uniformDistr)
inh_cells.initialize(v=uniformDistr)

exc_conn = p.FixedProbabilityConnector(pconn, rng=rng)
inh_conn = p.FixedProbabilityConnector(pconn, rng=rng)

connections = {
    'e2e': p.Projection(
        exc_cells, exc_cells, exc_conn, receptor_type='excitatory',
        synapse_type=p.StaticSynapse(weight=w_exc, delay=delay)),
    'e2i': p.Projection(
        exc_cells, inh_cells, exc_conn, receptor_type='excitatory',
        synapse_type=p.StaticSynapse(weight=w_exc, delay=delay)),
    'i2e': p.Projection(
        inh_cells, exc_cells, inh_conn, receptor_type='inhibitory',
        synapse_type=p.StaticSynapse(weight=w_inh, delay=delay)),
    'i2i': p.Projection(
        inh_cells, inh_cells, inh_conn, receptor_type='inhibitory',
        synapse_type=p.StaticSynapse(weight=w_inh, delay=delay))}

connections['ext2e'] = p.Projection(
    ext_stim, exc_cells, ext_conn, receptor_type='excitatory',
    synapse_type=p.StaticSynapse(weight=0.1))
connections['ext2i'] = p.Projection(
    ext_stim, inh_cells, ext_conn, receptor_type='excitatory',
    synapse_type=p.StaticSynapse(weight=0.1))

# === Setup recording ===
print("%s Setting up recording..." % node_id)
exc_cells.record("spikes")


# === Run simulation ===

p.run(tstop)
# === Finished with simulator ===




2022-01-10 15:28:53 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2022-01-10 15:28:53 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2022-01-10 15:28:53 INFO: Setting time scale factor to 1.
2022-01-10 15:28:53 INFO: Setting machine time step to 1000 micro-seconds.
2022-01-10 15:28:53 INFO: Simulating for 1000 1.0ms timesteps using a hardware timestep of 1000us
2022-01-10 15:28:53 INFO: Starting execution process


Detected PyNN version 0.9.4 and Neo version 0.6.1
1200 300
['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']
{'tau_m': 20.0, 'tau_syn_E': 5.0, 'tau_syn_I': 10.0, 'v_rest': -60.0, 'v_reset': -60.0, 'v_thresh': -50.0, 'cm': 0.2, 'tau_refrac': 5.0, 'i_offset': 0}
0 Setting up recording...


2022-01-10 15:28:56 INFO: Time 0:00:03.084393 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2022-01-10 15:29:04 INFO: Time 0:00:08.287314 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2022-01-10 15:29:09 INFO: Time 0:00:05.119121 taken by PartitionAndPlacePartitioner
Created spalloc job 6240786
2022-01-10 15:29:09 INFO: Created spalloc job 6240786
Waiting for board power commands to complete.
2022-01-10 15:29:09 INFO: Waiting for board power commands to complete.
2022-01-10 15:29:14 INFO: Time 0:00:05.047617 taken by SpallocAllocator
2022-01-10 15:29:14 INFO: Creating transceiver for 10.11.193.129
2022-01-10 15:29:14 INFO: Working out if machine is booted
2022-01-10 15:29:18 INFO: Attempting t

1000.0

In [2]:
# === Print results to file ===

exc_spikes = exc_cells.get_data("spikes")

Figure(
    # raster plot of the presynaptic neuron spike times
    Panel(exc_spikes.segments[0].spiketrains, xlabel="Time/ms", xticks=True,
          yticks=True, markersize=0.2, xlim=(0, tstop)),
    title="Vogels-Abbott benchmark: spikes",
    annotations="Simulated with {}".format(p.name())
)
plt.show()



Getting spikes for Excitatory_Cells
|0%                          50%                         100%|


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
p.end()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib
fig, ax = plt.subplots(1,3,figsize=(11,5))
ax = ax.flatten()
p=ax[0].hist(np.asarray(exc_obs.segments[0].analogsignals[0]).ravel(), 100)
p=ax[1].hist(np.asarray(exc_obs.segments[0].analogsignals[1]).ravel(), 100)
p=ax[2].hist(np.asarray(exc_obs.segments[0].analogsignals[2]).ravel(), 100)

p=ax[0].hist(np.asarray(inh_obs.segments[0].analogsignals[0]).ravel(), 100)
p=ax[1].hist(np.asarray(inh_obs.segments[0].analogsignals[1]).ravel(), 100)
p=ax[2].hist(np.asarray(inh_obs.segments[0].analogsignals[2]).ravel(), 100)
ax[2].set_xlim(0,0.05)
ax[1].set_xlim(0,0.3)

In [ ]:
import matplotlib.colors as colors
fig, ax = plt.subplots(2,3,figsize=(11,5))
ax = ax.flatten()
p=ax[0].matshow(np.asarray(exc_obs.segments[0].analogsignals[0]).T,  norm=colors.PowerNorm(gamma=1))
p=ax[1].matshow(np.asarray(exc_obs.segments[0].analogsignals[1]).T,  norm=colors.PowerNorm(gamma=0.25))
p=ax[2].matshow(np.asarray(exc_obs.segments[0].analogsignals[2]).T,  norm=colors.PowerNorm(gamma=0.25))
p=ax[3].matshow(np.asarray(inh_obs.segments[0].analogsignals[0]).T,  norm=colors.PowerNorm(gamma=0.25))
p=ax[4].matshow(np.asarray(inh_obs.segments[0].analogsignals[1]).T,  norm=colors.PowerNorm(gamma=0.25))
p=ax[5].matshow(np.asarray(inh_obs.segments[0].analogsignals[2]).T,  norm=colors.PowerNorm(gamma=0.25))

# study network oscillations

In [ ]:
%matplotlib
plt.plot(np.asarray(exc_obs.segments[0].analogsignals[0]).T[100])

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(11,5))
ax = ax.flatten()
p=ax[0].eventplot(stim_obs.segments[0].spiketrains)
p=ax[1].eventplot(exc_obs.segments[0].spiketrains)
p=ax[2].eventplot(inh_obs.segments[0].spiketrains)

In [ ]:
stim_spikes = []
for i in range(len(stim_obs.segments[0].spiketrains)):
    stim_spikes += [len(stim_obs.segments[0].spiketrains[i])]
    
exc_spikes = []
for i in range(len(exc_obs.segments[0].spiketrains)):
    exc_spikes += [len(exc_obs.segments[0].spiketrains[i])]
    
inh_spikes = []
for i in range(len(inh_obs.segments[0].spiketrains)):
    inh_spikes += [len(inh_obs.segments[0].spiketrains[i])]

fig, ax = plt.subplots(1,3,figsize=(11,5))
ax = ax.flatten()
p=ax[0].hist(stim_spikes)
p=ax[1].hist(exc_spikes)
p=ax[2].hist(inh_spikes)